In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import os
import geopandas as gpd
from shapely.geometry import LineString

from neo4j import GraphDatabase

PATH = Path.cwd().parent.joinpath('data')

## Goal

The goal of this notebook is to perform exploratory data analysis on the shadowfleet data and create a model for a graph database. 

### Data description

First we need to know if we have all necessary data and if not, what data we are missing. Our starting point is the shadowfleet data provided by the Kiev School of Economics Institute.

#### KSE

In [ ]:
kse = pd.read_csv(PATH.joinpath('processed', 'kse_shadowfleet.csv'),
                  parse_dates=['earliest_sanction_date'],
                  date_format=lambda x: pd.to_datetime(x, 
                                                       errors='coerce', 
                                                       format='%d-%m-%Y', 
                                                       dayfirst=True))

# create a set of imos
imos = set(kse.imo)

# Some vessels have multiple entries in the dataseet (because they carry multiple products)
# We will drop the duplicates

kse_dedup = kse.drop_duplicates(subset='imo')

print(f'''The KSE dataset contains {len(imos)} unique vessels
of which {kse[kse.earliest_sanction_date.notna()].imo.nunique()} are sanctioned.''')

In [ ]:
# Let's dive into the sanctions: when were they sanctioned?

sanctions = kse_dedup.groupby('earliest_sanction_date').imo.count().reset_index()

sanctions['earliest_sanction_date'] = pd.to_datetime(sanctions['earliest_sanction_date'], 
                                                     errors='coerce', 
                                                     format='%d-%m-%Y', dayfirst=True)

sanctions.earliest_sanction_date = pd.to_datetime(sanctions.earliest_sanction_date)
sanctions.sort_values('earliest_sanction_date', inplace=True)
sanctions.set_index('earliest_sanction_date', inplace=True, drop=True)

# Sanction dates

px.bar(sanctions,
       title='Sanction Dates',
       labels={'earliest_sanction_date':'Sanction Date'})

In [ ]:
# What is the Age of the shadowfleet?

px.histogram(kse_dedup, x='buildyear',
             title='Age of the Shadowfleet',
             labels={'buildyear':'Build Year'},
             )

In [ ]:
# What is the flag distribution? (N.B. the current flag is the flag as of 6th August 2024)

px.bar(kse_dedup.groupby('current_flag_06082024').imo.count()).\
       update_xaxes(categoryorder='total descending')

In [ ]:
# When dit the vessels enter the shadowfleet? (use our uninsured data)
# The first date should be discarded as it is the date of the first entry in the dataset, but the start of measurements

uninsured = pd.read_csv(PATH.joinpath('processed', 'uninsured.csv'),
                        parse_dates=['start_date', 'end_date', 'earliest_sanction_date'],
                        date_format='%Y-%m-%d')

px.bar(uninsured.groupby('start_date').imo.count(),
       title='Entry Dates of the Shadowfleet',
       labels={'start_date':'Entry Date'})

In [ ]:
# And what are the end dates? The last date (July 2024) should be discarded because it is not a real end date, but the end of measurements.

px.bar(uninsured.groupby('end_date').imo.count(),
       title='Exit Dates of the Shadowfleet',
       labels={'start_date':'Exit Date'})

#### Company histories and details

The equasis data consists of lists of companies owning or managing a vessel through time. That data is based on the IMO registry and could be very useful in finding relevant changes in vessel ownership. 

In [ ]:
companies = pd.read_csv(PATH.joinpath('processed', 'owners_companies.csv'))
companies[['start_date', 'end_date']] = companies[['start_date', 'end_date']].apply(pd.to_datetime)
len(companies)

In [ ]:
companies.head()

In [ ]:
# Are there any vessels in the shadowfleet that we don't have company information for?

print(f'There is {len(set(kse.imo).difference(set(companies.imo)))} imos not in the other dataset')

In [ ]:
# How many UNKNOWN companies are in this dataset?

companies[companies.company == 'UNKNOWN'].company.count()

This seems to be relevant. Let's see these missing values over time. If they are old, than its just an administrative issue. If the missing companies are more recent, then it would be interesting to know why these companies aren't known. The unknown companies are all ISM Managers.  

In [ ]:
unknown = companies[companies.company == 'UNKNOWN'].copy()
unknown.start_date = pd.to_datetime(unknown.start_date, errors='coerce', format='%Y-%m-%d', dayfirst=True).dt.year
px.histogram(unknown, x='start_date')

In [ ]:
# What kind of roles are unknown?

unknown.role.value_counts()

In [ ]:
# What companies had many ownership changes after the start of the Ukraine war?

companies.query('role == "Registered owner" & start_date > "01-01-2022"')\
         .groupby('imo')\
         .agg(ownership_changes=('company', 'nunique'))\
         .sort_values('ownership_changes', ascending=False)[0:20]

In [ ]:
# Let's look at the vessel with the most ownership changes after the Ukraine war

pd.merge(companies.query('imo == 9302126 & role == "Registered owner"'),
         kse[['imo', 'vessel_name']],
         on='imo',
         how='left')

In [ ]:
# When did ownership change?

px.bar(companies.groupby([pd.Grouper(key='end_date', freq='ME'), 'role'])\
                .agg(count=('role', 'count'))\
                .reset_index()\
                .set_index('end_date', drop=True).query('index > "01-01-2014"'),
       color='role',
       title='Ownership Changes Over Time',
       labels={'end_date':'End Date',
               'value': 'Number of Ownership Changes'})

In [ ]:
com_details = pd.read_csv(PATH.joinpath('processed', 'owners_companies_details.csv'))
com_details[['last_update', 'since']] = com_details[['last_update', 'since']].apply(pd.to_datetime)
com_details[['imo', 'company_imo', 'year_of_build']] = com_details[['imo', 'company_imo', 'year_of_build']].astype(int)
com_details.head()

### Events

Analyse port visits, loitering, AIS gaps and activity in known ship-to-ship transfer zones

In [268]:
ports = pd.read_parquet(PATH.joinpath('processed', 'ports.parquet'))
loitering = pd.read_parquet(PATH.joinpath('processed', 'loitering.parquet'))
ais = pd.read_parquet(PATH.joinpath('processed', 'ais.parquet'))
sts = pd.read_parquet(PATH.joinpath('processed', 'sts_tracks.parquet'))

#### Port visits

In [ ]:
# Clean ports

ports[['start', 'end']] = ports[['start', 'end']].apply(lambda x: pd.to_datetime(x).dt.date)

# See port visits over time

ports[ports.port_visit_startAnchorage_id.str.contains('tur-')].port_visit_startAnchorage_name.value_counts().nlargest(10)

In [252]:
ports['port_name'] = ports.groupby(['port_visit_startAnchorage_id'])['port_visit_startAnchorage_name'].ffill()

In [255]:
ports[ports.start > pd.to_datetime('2022-01-01').date()].to_csv(PATH.joinpath('processed', 'port_visits_2022.csv'), index=False)

In [ ]:
# Enter 3 iso country code and port name

COUNTRY = 'tur'
PORT = 'AMBARLI'

px.bar(ports[(ports.port_visit_startAnchorage_id.str.contains(f'{COUNTRY}-')) \
             & (ports.port_visit_startAnchorage_name==PORT)].groupby(pd.Grouper(key='start', freq='ME')).agg(visits=('ssvid', 'count')),
             title=f'Port visits from shadowfleet vessels to {PORT} by month',
             labels={'visits': 'Number of visits',
                     'start': 'Time'},)

#### Loitering

In [ ]:
loitering[['start', 'end']] = loitering[['start', 'end']].apply(pd.to_datetime)
loitering = loitering[loitering.start >= '2022-01-01'].copy()
loitering[['start', 'end']] = loitering[['start', 'end']].apply(lambda x: x.dt.date)
len(loitering)

In [273]:
loitering.to_csv(PATH.joinpath('processed', 'loitering_2022.csv'), index=False)

In [ ]:
loitering.head()

#### AIS

In [ ]:

ais['geometry'] = ais.apply(lambda x: LineString([(x['gap_offposition_lon'], x['gap_offposition_lat']) , (x['gap_onposition_lon'], x['gap_onposition_lat'])]), axis = 1)
ais = gpd.GeoDataFrame(ais, geometry='geometry', crs='EPSG:4326')

In [ ]:
ais[['geometry', 'name']].explore()

#### Ship-to-ship transfers

In [152]:
# Clean STS

sts.drop('seg_id', axis=1, inplace=True)
sts.timestamp = pd.to_datetime(sts.timestamp)
sts.query('timestamp > "2022-01-01"', inplace=True)
sts.sort_values('timestamp', inplace=True)



In [153]:
sts['week'] = sts.timestamp.dt.to_period('W').apply(lambda x: x.start_time)

In [ ]:
sts.groupby(['name', 'week']).agg(count=('name', 'count')).reset_index().sort_values('count', ascending=False).name.value_counts()[140:150]

In [ ]:
sts.columns

In [156]:
sts = gpd.GeoDataFrame(sts, geometry=gpd.points_from_xy(x=sts.lon, y=sts.lat), crs='EPSG:4326')
sts.week = sts.week.astype(str)



In [157]:
sts_locations = gpd.read_file(PATH.joinpath('geo', 'sts_locations.geojson'))
sts_locations.set_geometry('geometry', crs='EPSG:4326', inplace=True)
sts = gpd.sjoin(sts, sts_locations, how='left', predicate='within')

In [159]:
sts.drop('index_right', axis=1, inplace=True)
sts.rename(columns={'Name':'sts_location'}, inplace=True)

In [ ]:
sts.groupby('name').agg(count=('sts_location', 'nunique')).sort_values('count', ascending=False).head(10)

In [ ]:
sts.query('name=="AIDA"')[['geometry', 'week']].explore(column='week')

## Merge with helennic shipping data

In [4]:
hel = pd.read_csv(PATH.joinpath('processed', 'hellenic_shipping.csv'), sep=';')

In [ ]:
hel.head()

In [9]:
names = pd.read_csv(PATH.joinpath('processed', 'owners_names.csv'))

In [ ]:
set(kse.vessel_name.str.upper().str.replace(' ', '')).intersection(hel.NAME.str.replace(' ', '').str.upper())